In [1]:
import clip
import torch
import pandas as pd
from numba import cuda

In [2]:
device = cuda.get_current_device()
device.reset()

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

In [4]:
df=pd.read_pickle(r"C:\Users\KHU\Desktop\무신사\eval_data.pickle")

In [5]:
df.columns

Index(['image_link', 'processed_image'], dtype='object')

In [6]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [7]:
# 모델 로드
model.load_state_dict(torch.load("best_model.pt"))
model.eval()
model = model.to(device) # 모델을 디바이스로 이동

# 텍스트 입력
text = "A cute puppy playing in the park"

# 텍스트 인코딩
text_encoded = clip.tokenize([text]).to(device)
text_features = model.encode_text(text_encoded).float()

# 이미지 데이터셋 로드 및 특징 추출
title_list = df['image_link'].tolist()
image_processed = [img.to(device) for img in df['processed_image'].tolist()] # 이미지 텐서를 디바이스로 이동
image_features = []
cnt=0
for image in image_processed:
    image_features.append(model.encode_image(image).float())
    cnt+=1
    if(cnt>10):
        break

c:\Users\KHU\anaconda3\envs\clip\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


In [8]:
# 유사도 계산 및 최대값 찾기
similarities = [torch.cosine_similarity(text_features, img_feat, dim=-1) for img_feat in image_features]
max_idx = torch.argmax(torch.tensor(similarities))

# 가장 관련된 이미지 출력
print(f"Most relevant image: {title_list[max_idx]}")

Most relevant image: https://image.msscdn.net/images/style/detail/41650/detail_41650_6629b7f3484ee_500.jpg
